In [16]:
import pandas as pd
import numpy as np


In [17]:
import findspark
findspark.init()


In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, udf, lit

spark = SparkSession.builder.appName('read csv files').getOrCreate()


In [19]:
csv_df=spark.read.csv("NovemberTopics.csv",header='true')

In [20]:
Nov_topics=csv_df.toPandas()

In [21]:
Nov_topics.head()


,_c0,created_date,dominant_topic,dominant_topic_keywords,count
0,0,2022-12-18,topic3,"love, day, thank, celebrity, asian, acoty, jun...",56
1,1,2022-12-18,topic4,"life, new, world, never, yes, people, cup, mor...",236
2,2,2022-12-18,topic6,"morning, good, friend, happiness, right, eveni...",176
3,3,2022-12-18,topic2,"like, happy, much, birthday, love, best, well,...",287
4,4,2022-12-18,topic5,"god, respect, africa, mind, many, money, prote...",41


In [22]:
Nov_topics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   _c0                      126 non-null    object
 1   created_date             126 non-null    object
 2   dominant_topic           126 non-null    object
 3   dominant_topic_keywords  126 non-null    object
 4   count                    126 non-null    object
dtypes: object(5)
memory usage: 5.0+ KB


In [23]:
Nov_topics['created_date']=pd.to_datetime(Nov_topics['created_date'])
#Nov_topics['topic']=pd.to_datetime(Nov_topics['topic'])
Nov_topics['count']=pd.to_numeric(Nov_topics['count'])

In [24]:
df = Nov_topics.pivot_table(values='count', index=[
                            'created_date'], columns='dominant_topic_keywords')
df.head()


dominant_topic_keywords,"correct, best, twitter, mood, club, goat, see, king, remember, mil","god, respect, africa, mind, many, money, protect, arab, put, change","life, new, world, never, yes, people, cup, morocco, make, love","like, happy, much, birthday, love, best, well, back, feel, great","love, day, thank, celebrity, asian, acoty, junghoseok, acotyjunghoseok, netizensreport, darling","morning, good, friend, happiness, right, evening, thanks, day, take, night","photo, posted, let, english, video, know, free, hates, grave, check"
created_date,,,,,,,
2022-12-01,41,41,161,210,73,259,143
2022-12-02,84,53,142,221,81,274,133
2022-12-03,63,52,184,266,74,294,156
2022-12-04,61,33,150,217,160,329,133
2022-12-05,54,47,173,224,95,311,126


In [25]:
df.fillna(0, inplace=True)
df.sort_values(list(df.columns), inplace=True)
df = df.sort_index()


In [26]:
df.head()


dominant_topic_keywords,"correct, best, twitter, mood, club, goat, see, king, remember, mil","god, respect, africa, mind, many, money, protect, arab, put, change","life, new, world, never, yes, people, cup, morocco, make, love","like, happy, much, birthday, love, best, well, back, feel, great","love, day, thank, celebrity, asian, acoty, junghoseok, acotyjunghoseok, netizensreport, darling","morning, good, friend, happiness, right, evening, thanks, day, take, night","photo, posted, let, english, video, know, free, hates, grave, check"
created_date,,,,,,,
2022-12-01,41,41,161,210,73,259,143
2022-12-02,84,53,142,221,81,274,133
2022-12-03,63,52,184,266,74,294,156
2022-12-04,61,33,150,217,160,329,133
2022-12-05,54,47,173,224,95,311,126


In [27]:
df.iloc[:, 0:] = df.iloc[:, 0:].cumsum()
df.head()


dominant_topic_keywords,"correct, best, twitter, mood, club, goat, see, king, remember, mil","god, respect, africa, mind, many, money, protect, arab, put, change","life, new, world, never, yes, people, cup, morocco, make, love","like, happy, much, birthday, love, best, well, back, feel, great","love, day, thank, celebrity, asian, acoty, junghoseok, acotyjunghoseok, netizensreport, darling","morning, good, friend, happiness, right, evening, thanks, day, take, night","photo, posted, let, english, video, know, free, hates, grave, check"
created_date,,,,,,,
2022-12-01,41,41,161,210,73,259,143
2022-12-02,125,94,303,431,154,533,276
2022-12-03,188,146,487,697,228,827,432
2022-12-04,249,179,637,914,388,1156,565
2022-12-05,303,226,810,1138,483,1467,691


In [28]:

import bar_chart_race as bcr
bcr.__version__
import matplotlib as plt

In [30]:
bcr.bar_chart_race(df=df,
                   n_bars=7,
                   sort='desc',
                   title='tweets number under each topic in Nov',
                   filename="DecemeberTweets.mp4")


e:\Big Data\Twitter-Sentiment-Analysis\venv\lib\site-packages\bar_chart_race\_make_chart.py:889: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
e:\Big Data\Twitter-Sentiment-Analysis\venv\lib\site-packages\bar_chart_race\_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
e:\Big Data\Twitter-Sentiment-Analysis\venv\lib\site-packages\bar_chart_race\_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
